In [41]:
from wandb.integration.keras import WandbCallback
import tensorflow as tf
import tensorflow_datasets as tfds
import wandb

In [42]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [43]:
(train, val, test), data_info = tfds.load("malaria",with_info=True,
                                        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                        shuffle_files=True, as_supervised=True)

In [44]:

def resize_only(image, label):
    # Redimensionar
    image = tf.image.resize(image, (224, 224))
    # Normalizar (opcional pero recomendado)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (train
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

val_dataset = (val
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

test_dataset = (test
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

In [45]:
#inicializacion de wandb
wandb.init(project="test")
config = wandb.config
config.learning_rate = 1e-3
config.batch_size = 32
config.epochs = 20

In [46]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    tf.keras.layers.Conv2D(16,3,activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(config.learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:

train_batch = next(iter(train_dataset))

callbacks = [
    WandbCallback(
        save_model=False,
        log_weights=True,
        log_gradients=False,
        log_evaluation=True,
        training_data=train_batch,
    ),
    tf.keras.callbacks.ModelCheckpoint("ckpt_best.h5", save_best_only=True, monitor="val_loss")
]


In [48]:

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=config.epochs,
                    callbacks=callbacks)

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Epoch 1/20
689/689 [==============================] - 67s 97ms/step - loss: 0.8393 - accuracy: 0.6822 - val_loss: 0.5494 - val_accuracy: 0.7848
Epoch 2/20
Epoch 2/20


c:\Users\vpn\Documents\GitHub\Ciencia-de-datos\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


689/689 [==============================] - 66s 96ms/step - loss: 0.4202 - accuracy: 0.8500 - val_loss: 0.4257 - val_accuracy: 0.8400
Epoch 3/20
Epoch 3/20
689/689 [==============================] - 66s 96ms/step - loss: 0.3167 - accuracy: 0.8964 - val_loss: 0.4094 - val_accuracy: 0.8367
Epoch 4/20
Epoch 4/20
689/689 [==============================] - 67s 97ms/step - loss: 0.2582 - accuracy: 0.9196 - val_loss: 0.4218 - val_accuracy: 0.8295
Epoch 5/20
Epoch 5/20
689/689 [==============================] - 69s 100ms/step - loss: 0.2129 - accuracy: 0.9374 - val_loss: 0.4880 - val_accuracy: 0.7903
Epoch 6/20
Epoch 6/20
689/689 [==============================] - 68s 99ms/step - loss: 0.1818 - accuracy: 0.9486 - val_loss: 0.5432 - val_accuracy: 0.7729
Epoch 7/20
Epoch 7/20
689/689 [==============================] - 68s 98ms/step - loss: 0.1497 - accuracy: 0.9592 - val_loss: 0.5854 - val_accuracy: 0.7761
Epoch 8/20
Epoch 8/20
689/689 [==============================] - 68s 99ms/step - loss: 0.13

In [49]:
model.save("model_final.h5")
artifact = wandb.Artifact("malaria-cnn-model", type="model")
artifact.add_file("model_final.h5")
artifact.add_file("ckpt_best.h5")
wandb.log_artifact(artifact)
wandb.finish()

accuracy,▁▅▆▆▇▇▇▇████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▅██▇▅▅▅▅▅▃▃▅▃▂▄▃▁▁▂▂
val_loss,▂▁▁▁▁▂▂▂▃▃▄▄▅▅▄▆▇█▇█
accuracy,0.99211
best_epoch,2
best_val_loss,0.40944
epoch,19
loss,0.03328
val_accuracy,0.72025
